In [16]:
import veloxchem as vlx

In [17]:
class MockTask:
    def __init__(self, mol, basis, comm, ostream):

        self.molecule = mol
        self.ao_basis = basis
        self.mpi_comm = comm
        self.ostream = ostream

In [30]:
import adcc
from mpi4py import MPI
import sys
import numpy
numpy.set_printoptions(precision=5, suppress=True)

In [19]:
# Input settings
molecule_string = """
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037"""

basis_set_label = 'STO-3G'
scf_settings = {'conv_thresh': 1.0e-6}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-4, 'nstates': 3}

In [20]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [21]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           O           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000004704                         
                           H           0.896000004436        0.000000000000       -0.317000001569                         
                                                                                                                          
                

In [22]:
# SCF
task = MockTask(molecule, basis, comm, ostream)
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
scfdrv.task = task

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [23]:
# SCF first-order properties
scf_prop = vlx.ScfFirstOrderProperties(comm, ostream)
scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
scf_prop.print_properties(molecule)

                                                                                                                          
                                                Ground-State Dipole Moment                                                
                                               ----------------------------                                               
                                                                                                                          
                                   X   :         0.546314 a.u.         1.388593 Debye                                     
                                   Y   :        -0.000000 a.u.        -0.000000 Debye                                     
                                   Z   :         0.386302 a.u.         0.981881 Debye                                     
                                 Total :         0.669095 a.u.         1.700671 Debye                                     
                

In [10]:
# Linear Response
#lr_eig_drv = vlx.LinearResponseEigenSolver(comm, ostream)
#lr_eig_drv.update_settings(rsp_settings, method_settings)
#lr_results = lr_eig_drv.compute(molecule, basis, scfdrv.scf_tensors)
#lr_eig_drv.ostream.flush()

In [24]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 3                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [25]:
# Get eigenvalues and eigenvectors

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

#eig_vals = lr_results["eigenvalues"]
#eig_vecs = lr_results["eigenvectors"]

#eig_vec = eig_vecs[:, 0].copy() # first eigenvector
#print(eig_vec)

In [26]:
# Convert the first eigenvector to matrix form
#eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(eig_vec, nocc, nocc + nvir)

In [39]:
tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
#tda_size = tda_eig_vecs[:,0].shape
#tda_eig_vec = numpy.zeros(2*tda_size[0])
tda_eig_vec=tda_eig_vecs[:,0].copy()

In [40]:
print(tda_eig_vec)

[ 0.      -0.       0.      -0.       0.      -0.       0.      -0.
  1.      -0.00095]


In [34]:
tda_eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(tda_eig_vec, nocc, nocc + nvir)

In [35]:
print(tda_eig_vec_as_mat)

[[ 0.       0.       0.       0.       0.       0.      -0.     ]
 [ 0.       0.       0.       0.       0.       0.      -0.     ]
 [ 0.       0.       0.       0.       0.       0.      -0.     ]
 [ 0.       0.       0.       0.       0.       0.      -0.     ]
 [ 0.       0.       0.       0.       0.       1.      -0.00095]
 [ 0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.       0.     ]]


In [36]:
adc1 = adcc.run_adc(scfdrv, method='adc1', n_singlets=3, conv_tol=1e-4)

Starting adc1 singlet Jacobi-Davidson ...
Niter n_ss  max_residual  time  Ritz values
  1     6     0.0028135   22ms  [0.48344 0.5742  0.6052 ]
  2    10    2.4148e-29   46ms  [0.48344 0.5742  0.60214]
=== Converged ===
    Number of matrix applies:    10
    Total solver time:           70.512ms


In [37]:
adc1_eig_vec = adc1.excitation_vector[0]["s"].to_ndarray()
adc1_eig_vals = adc1.excitation_energy

In [38]:
print(adc1_eig_vec)
print(adc1_eig_vals)
print(adc1.describe())

[[-0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.     ]
 [-0.      -0.       0.       0.     ]
 [-0.       0.       0.       0.     ]
 [-0.70711  0.00067  0.       0.     ]
 [ 0.       0.      -0.       0.     ]
 [ 0.       0.       0.       0.     ]
 [ 0.       0.      -0.      -0.     ]
 [ 0.       0.      -0.       0.     ]
 [ 0.       0.      -0.70711  0.00067]]
[0.48344 0.5742  0.60214]
+----------------------------------------------------+
| adc1                          singlet ,  converged |
+----------------------------------------------------+
|  #        excitation energy     osc str    |v1|^2  |
|          (au)           (eV)                       |
|  0     0.4834361      13.15497   0.0038         1  |
|  1     0.5742004      15.62479   0.0000         1  |
|  2      0.602137      16.38498   0.0546         1  |
+----------------------------------------------------+



In [ ]:
print(tda_eig_vecs[:,0])
print(tda_eig_vals)

In [42]:
numpy.linalg.norm(tda_eig_vec)

0.9999999999999999